In [1]:
import numpy as np
import pandas as pd

import logging

# Hide logging messages.
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
%env CATALYST_LOG_LEVEL = 15
#!pip install tensorflow-macos
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel


env: CATALYST_LOG_LEVEL=15


In [8]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')
test_df = pd.read_csv('test.csv')
test_df
#set(val_df['author'].tolist()+train_df['author'].tolist())

,text,"Alger, Horatio, Jr.","Arthur, T. S. (Timothy Shay)","Ballantyne, R. M. (Robert Michael)","Bangs, John Kendrick","Barr, Amelia E.","Bennett, Arnold","Burnett, Frances Hodgson","Burroughs, John","Churchill, Winston",...,"Rohmer, Sax","Roy, Lillian Elizabeth","Shakespeare, William","Shoghi, Effendi","Smith, Francis Hopkinson","Spinoza, Benedictus de","Twain, Mark","Wallace, F. L. (Floyd L.)","Warner, Charles Dudley","Whittier, John Greenleaf"
0,a little princess by frances hodgson burnett a...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,john barleycorn by jack london chapter i it al...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,benedict de spinoza the ethics ethica ordine g...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,a team of about twenty project gutenberg volun...,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,the complete writings of charles dudley warner...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,mark twain a biography by albert bigelow paine...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,literature and life by william dean howells co...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,all for the best by arthur philadelphia conten...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,the crisis by winston churchill volume chapter...,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,this ebook was produced by david widger widger...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn import metrics
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

for param in model.parameters():
    param.requires_grad = False

#model.pooler = torch.nn.AdaptiveMaxPool1d(1)
# Replace the classification layer with a linear layer
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(0.1),
    torch.nn.Linear(model.config.hidden_size, 47)
)

# Unfreeze the weights of the last 4 layers
for param in model.encoder.layer[-4:].parameters():
    param.requires_grad = True

# Fine-tune the model on your downstream task
train_dataset = train_df
dev_dataset = val_df
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

model.to(device)
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [7]:
train_inputs = tokenizer(train_dataset['text'].tolist(),
                         padding=True,
                         truncation=True,
                         return_tensors='pt')
train_labels = torch.tensor(train_dataset.drop('text', axis=1).values)\
    .to(device, dtype = torch.float)
val_inputs = tokenizer(dev_dataset['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
val_labels = torch.tensor(dev_dataset.drop('text', axis=1).values)\
    .to(device, dtype = torch.float)
for epoch in range(1):
    model.train()
    outputs = model(train_inputs['input_ids'].to(device, dtype = torch.long),
                    attention_mask=train_inputs['attention_mask'].to(device, dtype = torch.long),
                    token_type_ids=train_inputs['token_type_ids'].to(device, dtype = torch.long))
    pooled_output= outputs.pooler_output
    # print(pooled_output.shape)
    # pooled_output = model.pooler(pooled_output.permute(0, 2, 1)).squeeze()
    logits = model.classifier(pooled_output)
    #print(logits.shape)
    loss_fn = torch.nn.BCEWithLogitsLoss()
    loss = loss_fn(logits, train_labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # Evaluate the model on the dev set
    with torch.no_grad():
        model.eval()
        outputs = model(val_inputs['input_ids'].to(device, dtype = torch.long),
                        attention_mask=val_inputs['attention_mask'].to(device, dtype = torch.long),
                        token_type_ids=val_inputs['token_type_ids'].to(device, dtype = torch.long))
        pooled_output = outputs.pooler_output
        # pooled_output = model.pooler(pooled_output.permute(0, 2, 1)).squeeze()
        logits = model.classifier(pooled_output)
        acc_outputs = np.array(torch.sigmoid(logits).cpu().detach().numpy().tolist()) >= 0.5
        acc = metrics.accuracy_score(val_labels.cpu().detach().numpy().tolist(),
                                     acc_outputs)
    print(f"Epoch {epoch + 1}:Loss:  {loss.item()}, Dev accuracy = {acc}")

# Use the fine-tuned model to make predictions on new data
test_dataset = test_df
with torch.no_grad():
    inputs = tokenizer(test_dataset['text'].tolist(), padding=True, truncation=True, return_tensors='pt')
    outputs = model(inputs['input_ids'].to(device, dtype = torch.long),
                    attention_mask=inputs['attention_mask'].to(device, dtype = torch.long),
                    token_type_ids=inputs['token_type_ids'].to(device, dtype = torch.long))
    pooled_output = outputs.pooler_output
    logits = model.classifier(pooled_output)
    preds = torch.argmax(logits, dim=1)
    test_dataset['pred'] = preds.tolist()

test_dataset['pred']


Epoch 1:Loss:  0.6936196088790894, Dev accuracy = 0.0


0     41
1     41
2     14
3     41
4     41
5     14
6     41
7     41
8     14
9     41
10    14
11    14
12    41
13    14
14    14
15     9
16    41
17    14
18    14
19    41
20    14
21    14
22    14
23    41
24    14
25     9
26    41
27    41
28    14
29    14
30    41
31    14
32    14
33    14
34    41
35     9
36    14
37    14
38    14
39    14
40    41
41    14
42    41
43    41
44    41
45    14
46    41
Name: pred, dtype: int64